In [1]:
import keras

Using TensorFlow backend.


In [1]:
import numpy as np
import pandas as pd
from keras.models import Model,Sequential
from keras.callbacks import TensorBoard
from keras.layers import Input, Dense, Reshape, Dropout, Activation,Flatten
from keras.optimizers import SGD, Adam
from sklearn.model_selection import train_test_split
from keras.models import model_from_json

Using TensorFlow backend.


In [2]:
transcoder =pd.read_csv('E:BDAP/Datasets/online_video_dataset/transcoding_mesurment.tsv', delimiter='\t', header=0)

In [3]:
transcoder.columns

Index(['id', 'duration', 'codec', 'width', 'height', 'bitrate', 'framerate',
       'i', 'p', 'b', 'frames', 'i_size', 'p_size', 'b_size', 'size',
       'o_codec', 'o_bitrate', 'o_framerate', 'o_width', 'o_height', 'umem',
       'utime'],
      dtype='object')

In [5]:
transcoder.shape

(68784, 22)

In [7]:
transcoder.values

array([['04t6-jw9czg', 130.35667, 'mpeg4', ..., 144, 22508, 0.612],
       ['04t6-jw9czg', 130.35667, 'mpeg4', ..., 240, 25164, 0.98],
       ['04t6-jw9czg', 130.35667, 'mpeg4', ..., 360, 29228, 1.216],
       ..., 
       ['zX17-vi0sqQ', 249.68, 'vp8', ..., 144, 88708, 0.752],
       ['zyiT-TzxIpk', 183.62333999999998, 'h264', ..., 240, 88724, 5.444],
       ['zZKo-QsY86U', 294.61334, 'mpeg4', ..., 144, 88736, 3.076]], dtype=object)

In [4]:
lr = 0.0004 ##### learning rate
wd = 0.01
num_classes = 2 #### output nodes
batch_size = 128
num_epochs = 5

In [5]:
def create_data():
    
    x = transcoder[['duration', 'width', 'height', 'bitrate', 'framerate',
       'i', 'p', 'b', 'frames', 'i_size', 'p_size', 'b_size', 'size',
        'o_bitrate', 'o_framerate', 'o_width', 'o_height', 'umem']]
    y = transcoder[['utime']]
    x_train, x_test, y_train, y_test = train_test_split(x,y)
    
    x_train = x_train.values
    y_train =y_train.values

    x_test = x_test.values
    y_test = y_test.values

    return x_train, y_train, x_test, y_test

In [6]:
def buildModel(x_train, y_train, x_test, y_test):

    input_img = Input(shape=(18,))

    dense_1 = Dense(512,activation='relu')(input_img)

    dense_2 = Dense(256, activation = 'relu')(dense_1)

    dense_3 = Dense(256, activation = 'relu')(dense_2)

    dense_4 = Dense(256, activation = 'relu')(dense_3)

    output = Dense(1, activation='relu')(dense_4)

    model = Model(input_img, output)

    # autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')
    sgd = SGD(lr=lr, decay=1e-3, momentum=0.9, nesterov=True, clipvalue=5.0)

    model.compile(loss='mean_squared_error', optimizer=sgd,metrics=['accuracy'])

    print (model.summary())
    
    model.fit(x_train, y_train, 
    nb_epoch=num_epochs, 
    batch_size=batch_size, 
    shuffle=True, 
    validation_data=(x_test, y_test))

    return model

In [7]:
def main():
    # getting the data
    x_train, y_train, x_test, y_test = create_data()

    model = buildModel(x_train, y_train, x_test, y_test)

    # decoded_imgs = autoencoder.predict(x_test)

    pass

In [8]:
if __name__ == '__main__':
    main()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 18)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               9728      
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_4 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 257       
Total params: 272,897
Trainable params: 272,897
Non-trainable params: 0
_________________________________________________________________
None

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:28: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 51588 samples, validate on 17196 samples
Epoch 1/5
51588/51588 [==============================] - 6s - loss: 366.2147 - acc: 0.0000e+00 - val_loss: 338.8464 - val_acc: 0.0000e+00

In [38]:
############### standardize

def create_data():
    trans1= pd.read_csv("E:BDAP/Datasets/online_video_dataset/transcoding_mesurment.tsv",delimiter='\t')

    trans=trans1.drop(['id','codec','o_codec'],axis=1)
    x=trans[['duration', 'width', 'height', 'bitrate', 'framerate', 'i', 'p', 'b',
       'frames', 'i_size', 'p_size', 'b_size', 'size', 'o_bitrate',
       'o_framerate', 'o_width', 'o_height', 'umem']]
    y=trans['utime']

    x_train,x_test,y_train,y_test=train_test_split(x,y)
    x_train=x_train.values
    y_train=y_train.values
    x_test=x_test.values
    y_test=y_test.values
    x_train = (x_train - np.mean(x_train,axis=0)) / np.std(x_train)
    x_test= (x_test - np.mean(x_test,axis=0)) / np.std(x_test)
    y_train = (y_train - np.mean(y_train,axis=0)) / np.std(y_train)
    y_test = (y_test - np.mean(y_test,axis=0)) / np.std(y_test)
    
    return x_train, y_train, x_test, y_test

In [39]:
def buildModel(x_train, y_train, x_test, y_test):

    input_img = Input(shape=(18,))

    dense_1 = Dense(512,activation='relu')(input_img)

    dense_2 = Dense(256, activation = 'relu')(dense_1)

    dense_3 = Dense(256, activation = 'relu')(dense_2)

    dense_4 = Dense(256, activation = 'relu')(dense_3)

    output = Dense(1, activation='relu')(dense_4)

    model = Model(input_img, output)

    # autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')
    sgd = SGD(lr=lr, decay=1e-3, momentum=0.9, nesterov=True, clipvalue=5.0)

    model.compile(loss='mean_squared_error', optimizer=sgd,metrics=['accuracy'])

    print (model.summary())
    
    model.fit(x_train, y_train, 
    nb_epoch=num_epochs, 
    batch_size=batch_size, 
    shuffle=True, 
    validation_data=(x_test, y_test))

    return model

In [40]:
def main():
    # getting the data
    x_train, y_train, x_test, y_test = create_data()

    model = buildModel(x_train, y_train, x_test, y_test)

    # decoded_imgs = autoencoder.predict(x_test)

    pass
if __name__ == '__main__':
    main()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 18)                0         
_________________________________________________________________
dense_26 (Dense)             (None, 512)               9728      
_________________________________________________________________
dense_27 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_28 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_29 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_30 (Dense)             (None, 1)                 257       
Total params: 272,897
Trainable params: 272,897
Non-trainable params: 0
_________________________________________________________________
None

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:28: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 51588 samples, validate on 17196 samples
Epoch 1/5
51588/51588 [==============================] - 6s - loss: 0.9921 - acc: 0.0000e+00 - val_loss: 0.9941 - val_acc: 0.0000e+00

In [ ]:
#################  Classifier

In [41]:
transcoder['class'] = 'High'
transcoder['class'][transcoder['utime']>np.mean(transcoder['utime'])] = 1
transcoder['class'][transcoder['utime']<np.mean(transcoder['utime'])] = 0
transcoder.columns

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


Index(['id', 'duration', 'codec', 'width', 'height', 'bitrate', 'framerate',
       'i', 'p', 'b', 'frames', 'i_size', 'p_size', 'b_size', 'size',
       'o_codec', 'o_bitrate', 'o_framerate', 'o_width', 'o_height', 'umem',
       'utime', 'class'],
      dtype='object')

In [42]:
def create_data():
    
    x = transcoder[['duration', 'width', 'height', 'bitrate', 'framerate',
       'i', 'p', 'b', 'frames', 'i_size', 'p_size', 'b_size', 'size',
        'o_bitrate', 'o_framerate', 'o_width', 'o_height', 'umem']]
    y = transcoder[['class']]
    x_train, x_test, y_train, y_test = train_test_split(x,y)
    
    x_train = x_train.values
    y_train =y_train.values

    x_test = x_test.values
    y_test = y_test.values
    
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)

    return x_train, y_train, x_test, y_test

In [43]:
def buildModel(x_train, y_train, x_test, y_test):

    input_img = Input(shape=(18,))

    dense_1 = Dense(512,activation='relu')(input_img)

    dense_2 = Dense(256, activation = 'relu')(dense_1)

    dense_3 = Dense(256, activation = 'relu')(dense_2)

    dense_4 = Dense(256, activation = 'relu')(dense_3)

    output = Dense(2, activation='sigmoid')(dense_4)

    model = Model(input_img, output)
    

    model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

    print (model.summary())
    
    model.fit(x_train, y_train, 
    nb_epoch=num_epochs, 
    batch_size=batch_size, 
    shuffle=True, 
    validation_data=(x_test, y_test))

    return model

In [44]:
def main():
    # getting the data
    x_train, y_train, x_test, y_test = create_data()

    model = buildModel(x_train, y_train, x_test, y_test)

    # decoded_imgs = autoencoder.predict(x_test)

    pass

In [45]:
if __name__ == '__main__':
    main()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 18)                0         
_________________________________________________________________
dense_31 (Dense)             (None, 512)               9728      
_________________________________________________________________
dense_32 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_33 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_34 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_35 (Dense)             (None, 2)                 514       
Total params: 273,154
Trainable params: 273,154
Non-trainable params: 0
_________________________________________________________________
None

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:28: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 51588 samples, validate on 17196 samples
Epoch 1/5
51588/51588 [==============================] - 6s - loss: 4.2310 - acc: 0.7361 - val_loss: 4.1595 - val_acc: 0.7406